This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/pick.html).  I recommend having both windows open, side-by-side!

In [1]:
import numpy as np
from IPython.display import clear_output
from pydrake.all import (
    AbstractValue,
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    JointSliders,
    LeafSystem,
    MeshcatVisualizer,
    Parser,
    RigidTransform,
    RollPitchYaw,
    StartMeshcat,
)

from manipulation import ConfigureParser, running_as_notebook
from manipulation.scenarios import AddMultibodyTriad

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


# Forward kinematics of the UR5e TCP frame

This example demonstrates forward kinematics with the custom UR5e robot arm and NetFT probe end-effector. Move the joint sliders in the MeshCat visualizer to see how the TCP (tool center point) pose changes.

If you click on the "Open Controls" menu in the MeshCat visualizer, and dig into the menu `meshcat->drake->Source` then you will see elements for the UR5e robot and the NetFT probe end-effector. You can enable/disable their visualization. Give it a spin!

In [ ]:
class PrintTCPPose(LeafSystem):
    def __init__(self, body_index):
        LeafSystem.__init__(self)
        self._body_index = body_index
        self.DeclareAbstractInputPort(
            "body_poses", AbstractValue.Make([RigidTransform()])
        )
        self.DeclareForcedPublishEvent(self.Publish)

    def Publish(self, context):
        pose = self.get_input_port().Eval(context)[self._body_index]
        print(pose)
        print(
            "tcp position (m): "
            + np.array2string(
                pose.translation(),
                formatter={"float": lambda x: "{:3.2f}".format(x)},
            )
        )
        print(
            "tcp roll-pitch-yaw (rad): "
            + np.array2string(
                RollPitchYaw(pose.rotation()).vector(),
                formatter={"float": lambda x: "{:3.2f}".format(x)},
            )
        )
        clear_output(wait=True)


def ur5e_tcp_forward_kinematics_example():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0)
    parser = Parser(plant)
    ConfigureParser(parser)
    # Register UR5e package
    parser.package_map().Add('ur5e_description', '/home/ajay/drake/ur5e_description')
    # Load composite model (creates 'ur5e' and 'netft_probe' model instances)
    parser.AddModels('/home/ajay/drake/ur5e_description/ur5e_netft_probe.dmd.yaml')
    plant.Finalize()

    # Draw the frames
    # UR5e robot links
    ur5e_frames = [
        "base_link_inertia",
        "shoulder_link",
        "upper_arm_link",
        "forearm_link",
        "wrist_1_link",
        "wrist_2_link",
        "wrist_3_link",
        "flange",
    ]
    for frame_name in ur5e_frames:
        AddMultibodyTriad(plant.GetFrameByName(frame_name), scene_graph)

    # End-effector frames
    netft_probe_frames = ["netft_link", "p42v_link", "tcp"]
    for frame_name in netft_probe_frames:
        AddMultibodyTriad(plant.GetFrameByName(frame_name), scene_graph)

    meshcat.Delete()
    meshcat.DeleteAddedControls()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph.get_query_output_port(), meshcat
    )

    netft_probe_instance = plant.GetModelInstanceByName("netft_probe")
    tcp_body = plant.GetBodyByName("tcp", netft_probe_instance)
    print_tcp_pose = builder.AddSystem(PrintTCPPose(tcp_body.index()))
    builder.Connect(plant.get_body_poses_output_port(), print_tcp_pose.get_input_port())

    default_interactive_timeout = None if running_as_notebook else 1.0
    sliders = builder.AddSystem(JointSliders(meshcat, plant))
    diagram = builder.Build()
    sliders.Run(diagram, default_interactive_timeout)
    meshcat.DeleteAddedControls()


ur5e_tcp_forward_kinematics_example()

INFO:drake:Press the 'Stop JointSliders' button in Meshcat or press 'Escape' to continue.


RigidTransform(
  R=RotationMatrix([
    [-1.0, 1.8369701992233881e-16, 6.123233998248561e-17],
    [6.123234002016247e-17, 2.0510330605065122e-10, 1.0],
    [1.8369701990977985e-16, 1.0, -2.0510330605065122e-10],
  ]),
  p=[0.8171999999999999, 0.5086279999591022, 0.06279999989567868],
)
tcp position (m): [0.82 0.51 0.06]
tcp roll-pitch-yaw (rad): [1.57 -0.00 -3.14]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=05e7ca3d-5650-42c9-9342-60bc45e7d545' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>